In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
import pickle
import seaborn as sns
%matplotlib inline

In [351]:
#grab html from imdb
url = 'http://www.imdb.com/chart/top?ref_=nv_mv_250_6'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [352]:
#create 9 pages worth of html files for each genre
genre_links = soup.find(class_='quicklinks').find_all('a')
second_page_suffix = 'page=2&ref_=adv_nxt'
third_page_suffix = 'page=3&ref_=adv_nxt'
fourth_page_suffix = 'page=4&ref_=adv_nxt'
fifth_page_suffix = 'page=5&ref_=adv_nxt'
sixth_page_suffix = 'page=6&ref_=adv_nxt'
seventh_page_suffix = 'page=7&ref_=adv_nxt'
eighth_page_suffix = 'page=8&ref_=adv_nxt'
ninth_page_suffix = 'page=9&ref_=adv_nxt'

genres = []
for link in genre_links:
    genre_url = 'http://www.imdb.com' + str(link['href'])
    genre_url2 = genre_url[:len(genre_url)-16] + second_page_suffix
    genre_url3 = genre_url[:len(genre_url)-16] + third_page_suffix
    genre_url4 = genre_url[:len(genre_url)-16] + fourth_page_suffix
    genre_url5 = genre_url[:len(genre_url)-16] + fifth_page_suffix
    genre_url6 = genre_url[:len(genre_url)-16] + sixth_page_suffix
    genre_url7 = genre_url[:len(genre_url)-16] + seventh_page_suffix
    genre_url8 = genre_url[:len(genre_url)-16] + eighth_page_suffix
    genre_url9 = genre_url[:len(genre_url)-16] + ninth_page_suffix
    
    
    genre = str(link.text)
    genres.append(genre)
    
    genre_response = requests.get(genre_url)
    genre_response2 = requests.get(genre_url2)
    genre_response3 = requests.get(genre_url3)
    genre_response4 = requests.get(genre_url4)
    genre_response5 = requests.get(genre_url5)
    genre_response6 = requests.get(genre_url6)
    genre_response7 = requests.get(genre_url7)
    genre_response8 = requests.get(genre_url8)
    genre_response9 = requests.get(genre_url9)

    
    with open("./HTML_files_genres/" + genre + "1.html", 'wb') as f:
        f.write(genre_response.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "2.html", 'wb') as f:
        f.write(genre_response2.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "3.html", 'wb') as f:
        f.write(genre_response3.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "4.html", 'wb') as f:
        f.write(genre_response4.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "5.html", 'wb') as f:
        f.write(genre_response5.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "6.html", 'wb') as f:
        f.write(genre_response6.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "7.html", 'wb') as f:
        f.write(genre_response7.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "8.html", 'wb') as f:
        f.write(genre_response8.text.encode('utf-8'))
    with open("./HTML_files_genres/" + genre + "9.html", 'wb') as f:
        f.write(genre_response9.text.encode('utf-8'))

In [353]:
#grab links from page and create html for each movie
movies = []
for genre in genres:
    for i in range(1,10):
        with open('./HTML_files_genres/' + genre + str(i) + '.html','r') as genre_page:
            genre_soup = BeautifulSoup(genre_page, 'lxml')
            movie_blocks = genre_soup.find_all('h3',attrs={"class":"lister-item-header"})

            for link in movie_blocks:
                movie_url = 'http://www.imdb.com' + str(link.find('a')['href'])
                movie = str(link.find('a').text)
                if movie not in movies:
                    movies.append(movie)
                    movie_response = requests.get(movie_url)
                    movie = movie.replace("/", "-")
                    with open("./HTML_files_movies/" + movie + ".html", 'wb') as f:
                        f.write(movie_response.text.encode('utf-8'))


In [371]:
def grab_title_year(movie_soup):
    title_year = movie_soup.find('h1', attrs = {'itemprop':'name'})
    title = ''
    year = ''
    if title_year:
        title = movie_soup.find('h1', attrs = {'itemprop':'name'}).text[:-8]
        year = movie_soup.find('h1', attrs = {'itemprop':'name'}).text[-6:-2]
    return [title, year]

def grab_month(movie_soup):
    month = np.nan
    release_date = (movie_soup.find('h4', {'class':'inline'}, text=re.compile('Release Date')))
    if release_date:
        release_date = release_date.next_sibling.replace('\n', '')
        month = release_date.split()[1]
    return month

def grab_imdb_score(movie_soup):
    imdb_score = movie_soup.find('div', attrs = {'class':'ratingValue'})
    if imdb_score:
        imdb_score = float(imdb_score.find('span').text)
    return imdb_score

def grab_rating(movie_soup):
    rating = movie_soup.find('meta', {'itemprop': 'contentRating'})
    if rating: 
        rating = rating['content']
    return rating
        
def grab_runtime(movie_soup):
    runtime = 0
    runtime_object = movie_soup.find('time', {'itemprop': 'duration'})#['datetime']
    if runtime_object: 
        runtime = runtime_object['datetime'][2:-1]
    return runtime

def grab_genres(movie_soup):
    genre_info = movie_soup.find('div', {'class': 'subtext'})
    genre1 = ''
    genre2 = np.nan
    if genre_info:
        genre_info = movie_soup.find('div', {'class': 'subtext'}).find_all('a')
        genre_item1 = genre_info[0].find('span', {'itemprop': 'genre'})
        genre1 = genre_item1.text
        genre_item2 = genre_info[1].find('span', {'itemprop': 'genre'})
        #genre2 = ''
        if genre_item2:
            genre2 = genre_item2.text
    return [genre1, genre2]

def grab_metascore(movie_soup):
    metascore = movie_soup.find('div', attrs = {'class': re.compile('metacriticScore score_favorable.*')})
    if metascore:
        metascore = int(metascore.find('span').text)
    return metascore

def grab_imdb_reviews(movie_soup):
    imdb_reviews = movie_soup.find('span', attrs = {'itemprop':'ratingCount'})
    if imdb_reviews:
        imdb_reviews = int(imdb_reviews.text.replace(",", ""))
    return imdb_reviews

def grab_oscars(movie_soup):
    oscar_info = movie_soup.find('span', {'itemprop':'awards'})
    if oscar_info:
        oscar_info = movie_soup.find('span', {'itemprop':'awards'}).text.replace('\n', '').strip()
        if 'Won' in oscar_info:
            return int(re.findall('\d+', oscar_info)[0])
    return 0
    

def grab_monetary_figures(movie_soup):
    budget = (movie_soup.find('h4', {'class':'inline'}, text=re.compile('Budget')))
    if budget:          
        budget = budget.next_sibling.replace(' ','').strip('\n').replace('$','').replace(',','')

    opening_week = (movie_soup.find('h4', {'class':'inline'}, text=re.compile('Opening Weekend')))
    if opening_week:            
        opening_week = opening_week.next_sibling.replace(' ','').strip('\n').replace('$','').replace(',','')
        opening_week = opening_week.split('(')[0].strip('\n')

    domestic_total_gross = (movie_soup.find('h4', {'class':'inline'}, text=re.compile('Gross')))
    if domestic_total_gross:
        domestic_total_gross = domestic_total_gross.next_sibling.replace(' ','').strip('\n').replace('$','').replace(',','')
    
    return [budget, opening_week, domestic_total_gross]

In [372]:
movie_data = []
headers = ['Title',
           'Year',
           'Month',
           'IMDB_Score',
           'Rating',
           'Run_Time',
           'Genre1',
           'Genre2',
           'Metascore',
           'IMDB_Reviews',
           'Oscar_Wins',
           'Budget',
           'Opening_Week_Revenue',
           'Gross_Revenue']

for movie in movies:
    movie = movie.replace("/", "-") #for edge case when title has /, so file reader doesn't think it's a directory
    with open('./HTML_files_movies/' + movie + '.html', 'r') as movie_page:
        
        movie_soup = BeautifulSoup(movie_page, 'lxml')

        title_year = grab_title_year(movie_soup)
        title = title_year[0]
        year = title_year[1]
        month = grab_month(movie_soup)
        imdb_score = grab_imdb_score(movie_soup)
        imdb_reviews = grab_imdb_reviews(movie_soup)
        rating = grab_rating(movie_soup)
        runtime = grab_runtime(movie_soup)

        genre_info = grab_genres(movie_soup)
        genre1 = genre_info[0]
        genre2 = genre_info[1]

        metascore = grab_metascore(movie_soup)
        oscar_wins = grab_oscars(movie_soup)

        monetary_figs = grab_monetary_figures(movie_soup)
        budget = monetary_figs[0]
        opening_week = monetary_figs[1]
        domestic_total_gross = monetary_figs[2]

        movie_dict = dict(zip(headers, [title,
                                        year,
                                        month,
                                        imdb_score,
                                        rating,
                                        runtime,
                                        genre1,
                                        genre2,
                                        metascore,
                                        imdb_reviews,
                                        oscar_wins,
                                        budget,
                                        opening_week,
                                        domestic_total_gross]))
        
        movie_data.append(movie_dict)

In [373]:
movie_data

[{'Budget': '185000000',
  'Genre1': 'Action',
  'Genre2': 'Crime',
  'Gross_Revenue': '533316061',
  'IMDB_Reviews': 1698798,
  'IMDB_Score': 9.0,
  'Metascore': 82,
  'Month': 'July',
  'Opening_Week_Revenue': '158411483',
  'Oscar_Wins': 2,
  'Rating': 'PG-13',
  'Run_Time': '152',
  'Title': 'The Dark Knight',
  'Year': '2008'},
 {'Budget': '94000000',
  'Genre1': 'Action',
  'Genre2': 'Adventure',
  'Gross_Revenue': '377019252',
  'IMDB_Reviews': 1230627,
  'IMDB_Score': 8.9,
  'Metascore': 94,
  'Month': 'December',
  'Opening_Week_Revenue': '£15021761',
  'Oscar_Wins': 11,
  'Rating': 'PG-13',
  'Run_Time': '201',
  'Title': 'The Lord of the Rings: The Return of the King',
  'Year': '2003'},
 {'Budget': '160000000',
  'Genre1': 'Action',
  'Genre2': 'Adventure',
  'Gross_Revenue': '292568851',
  'IMDB_Reviews': 1489333,
  'IMDB_Score': 8.8,
  'Metascore': 74,
  'Month': 'July',
  'Opening_Week_Revenue': '£5912814',
  'Oscar_Wins': 4,
  'Rating': 'PG-13',
  'Run_Time': '148',
  '

In [374]:
import pickle

with open('movie_data.pkl', 'wb') as picklefile:
    pickle.dump(movie_data, picklefile)